# Tutorial - Programming Z3
Available at http://theory.stanford.edu/~nikolaj/programmingz3.html 

## Chapter 2 - Logical Interfaces to Z3

##### Monday, March28, 2022

This notebook was created using _z3-solver 4.8.15.0_

### Import Z3
##### The first time, you will need to install z3-solver

```pip install z3-solver```

I chose to do that from the `cmd.exe` prompt that is available from Anaconda. I did it there, rather than here in Jupyter, because it will only need to be done once to make it available to all Jupyter notebooks. Now the `import` will work.

In [161]:
from z3 import *

And that makes Z3 available for this whole Jupyter notebook.

### Run Z3 from Python
Here is a quick example. The variables on the left-hand side are Python variables. On the right-hand side the Z3 variables are created, with the names separated by spaces.

In [162]:
Tie, Shirt = Bools('Tie Shirt')
s = Solver()
s.add(
    Or(Tie, Shirt), 
    Or(Not(Tie), Shirt), 
    Or(Not(Tie), Not(Shirt)))
print(s.check())
print(s.model())

sat
[Tie = False, Shirt = True]


In logic, the word "model" refers to a solution, an assignment of values to all variables such that the statement is true. So if `[Tie = False, Shirt = True]` is the solution found for a set of statements, then it is a model for them. 

Note that the solution values from within the solver are not assigned to the Python variables _Tie_ and _Shirt_.

In [163]:
print(Tie)
print(Shirt)
print(Or(Tie,Shirt)) # Tie and Shirt have not even been assigned values for use inside Z2 expressions

Tie
Shirt
Or(Tie, Shirt)


In [164]:
print(  type(Tie),   Tie.sort(),  Tie.decl()  )

<class 'z3.z3.BoolRef'> Bool Tie


\
Next we do the same thing as above using the `solve` function, which combines the process of setting up a solver, then performing an `add`, a `check`, and printing the `model`. (The variables Tie and Shirt still exist from the cell above, and they have not been fixed at the values in the above solution, so they are available for reuse in a Z3 expression.)

In [165]:
solve(
    Or(Tie, Shirt), 
    Or(Not(Tie), Shirt), 
    Or(Not(Tie), Not(Shirt))
    )


[Tie = False, Shirt = True]


\
Next we attempt to solve a more complicated formula, and we see what is reported when there is no solution.

In [166]:
Z = IntSort()
f = Function('f', Z, Z)
x, y, z = Ints('x y z')
A = Array('A', Z, Z)
fml = Implies(x + 2 == y, f(Store(A, x, 3)[y - 2]) == f(y - x + 1))
solve(Not(fml))

no solution


The Tutorial text gives the explanation for why fml is always true and, therefore, the solve fails to find a solution.

##### SMT-LIB - Satisfiability Modulo Theories
SMT-LIB is a format for logic that includes complex theories, for instance, the integers. Here is an example formula.

```
 (set-logic QF_LIA)
 (declare-const x Int)
 (declare-const y Int)
 (assert (> (+ (mod x 4) (* 3 (div y 2))) (- x y)))
 (check-sat)
```

And here is the corresponding Z3 in Python:

In [167]:
x, y = Ints('x y')
solve((x % 4) + 3 * (y / 2) > x - y)

[x = 0, y = 1]


\
We can see the equivalent SMT-LIB2 representation of our Python code by printing `solver.sexpr()`. In the following, we are using the same _x_ and _y_ declared immediately above, but with a different solver. (And notice how _x_ and _y_ are declared in the SMT output.)

In [168]:
s = Solver()
s.add((x % 4) + 3 * (y / 2) > x - y)
print(s.sexpr())

(declare-fun y () Int)
(declare-fun x () Int)
(assert (> (+ (mod x 4) (* 3 (div y 2))) (- x y)))



The nullary function `(declare-fun y () Int)` is the SMT declaration of the constant `y` corresponding to the declaration 

`x, y = Ints('x y')`

## \*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*
## \*\*\* WORKING HERE TO CREATE CORRESPONDING JAVA CODE \*\*\*
## \*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*\*



## 2.1. Sorts

Above, we've seen examples of booleans and integers:

```
Tie, Shirt = Bools('Tie Shirt')
x, y, z = Ints('x y z')
```

and arrays with integer indices and integer elements:

```
A = ('A', Z, Z)
```

There are also _Real_, _String_, _(\_ BitVec n)_ for every bit-width _n_, and (_Seq S_) for every sort _S_.

In [169]:
Tie, Shirt = Bools('Tie Shirt')
i,j,k = Ints('i j k')
Z = IntSort()
A = Array('A', Z, Z)
Select(A, i)
A[i]
Store(A, i, 3)

x,y = Reals('x y')
s1, s2 = Strings('s1 s2')
u, v = BitVecs('u v', 32)
s, t = Consts('s t', SeqSort(IntSort()))

There is also the ability to declare new sorts with _DeclareSort_. Note that sorts (types) are never empty. Thus the following is unsatisfiable (because `ForAll(s, False)` is only true if no _s_ exists).

In [170]:
S = DeclareSort('S')
s = Const('s', S)
solve(ForAll(s, s != s))
solve(ForAll(s, False))

# What about the opposite
solve(Exists(s, s == s))
solve(Exists(s, True))

no solution
no solution
[]
[]


##### $\color{red}{\text{WHAT?}}$

That last result is strange. Maybe it means "There are no contradictions, but examples do not make sense in this context." Or perhaps, "Unrestricted examples! Everything is a solution!".

$\color{red}{\text{TODO: Come back to this when I know more about how Z3 reports results.}}$

## 2.2. Signatures

Here is a declaration of a function from integers to integers:

```
Z = IntSort()
f = Function('f', Z, Z)
```

Constants are nullary functions. 

Functions with a boolean range can be used to create formulas. The function _g_ in Section 2.3 below is an example.

## 2.3 Terms and Formulas
Functions with a boolean co-domain can be used with other functions, and in addition can be used in assertions or submitted for solution.

In [171]:
B = BoolSort()
f = Function('f', B, Z)
g = Function('g', Z, B)
a = Bool('a')
solve(g(1+f(a)))

[a = False, f = [else -> 0], g = [else -> True]]


The `f = [else -> 0]` indicates that the proposed solution for the function _f_ only has the default case and thus maps everything to zero.

\
The standard logical operators are available: `And`, `Or`, `Implies`, `Not`, and `Xor`. For the biconditional operator, the Python API uses an equality `==` between 
variables that are assigned truth values.
(There is probably a separate method that serves as the biconditional operator in Java. I don't think that the .equals() method would be used for that because that would make it return a Z3 value rather than a Java boolean.)

\
Functions can be analyzed. Here we create the function `n` by assigning an expression over variables to the variable
name `n`. 


In [172]:
x = Int('x')
y = Int('y')
n = x + y >= 3
print("n string    ", n.__str__())
print("n sort      ", n.sort())
print("num args:   ", n.num_args())
print("children:   ", n.children())
print("1st child:  ", n.arg(0))
print("n arg0 sort ", n.arg(0).sort())
print("2nd child:  ", n.arg(1))
print("n arg1 sort ", n.arg(1).sort())
print("n arg1 doc  ", n.arg(1).__doc__)
print("operator:   ", n.decl())
print("op name:    ", n.decl().name())

n string     x + y >= 3
n sort       Bool
num args:    2
children:    [x + y, 3]
1st child:   x + y
n arg0 sort  Int
2nd child:   3
n arg1 sort  Int
n arg1 doc   Integer values.
operator:    >=
op name:     >=


## 2.4. Quantifiers and Lambda binding

Z3 has a notation for universal and existential quantifiers, which binds names as new variables within the scope of the quantifer.

In [173]:
solve([y == x + 1, ForAll([y], Implies(y <= 0, x < y))])
solve([y == x + 1, ForAll([y], Implies(y <= 0, x > y))])

no solution
[x = 2, y = 3]


In [174]:
solve([y == x + 1, Exists([y], Implies(y <= 0, x < y))])
solve([y == x + 1, Exists([y], And(x > y, x <= 0, y >= 0))])

[x = 0, y = 1]
no solution


\
Wierdly, DeMorgan does not hold for quantifiers.

In [175]:
b = Function("b", Z, B)
prove( ForAll([x],b(x)) == Not(Exists([x],b(x))) )

counterexample
[b = [else -> False]]


### Problem with quantification???
And my exploration of this leads to more odd results.
\
\
***NEED TO:*** $\color{red}{\text{Check Gries & Schneider, then submit a GitHub issue.}}$

In [176]:
solve(ForAll(s, s != s))
prove(ForAll(s, s != s))
print("Shouldn't that lack of counter examples mean proved?\n")

print("But here is another example that goes the other way:")
prove(ForAll(s, False) == False ) 
prove(ForAll(s, False))
print("Here the lack of counter examples cannot mean proved.")

no solution
counterexample
[]
Shouldn't that lack of counter examples mean proved?

But here is another example that goes the other way:
proved
counterexample
[]
Here the lack of counter examples cannot mean proved.


In [177]:
b = Function("b", Z, B)
prove( ForAll([x], False) == False ) 
prove(     Exists([x], False) == False  )
prove( Not(Exists([x], False) == False) )
print("Here the lack of counter examples does not mean proved!\n")

prove(Not(Exists([x], False)))
prove(Not(Exists([x], False) == True))
print("Here the lack of counter examples does not mean proved!\n")

F = False
F == F
solve(    ForAll([x], False)  ==     Exists([x], False)  )
solve(    ForAll([x], False)  == Not(Exists([x], False)) )
solve( eq(ForAll([x], False),        Exists([x], False)) ) 
solve( eq(ForAll([x], False),    Not(Exists([x], False))))

prove( ForAll([x], False) ==     Exists([x], False)  )
prove( ForAll([x], b(x) ) == Not(Exists([x], b(x) )) )
ForAll([x], False) == Not(Exists([x], False))

proved
proved
counterexample
[]
Here the lack of counter examples does not mean proved!

proved
proved
Here the lack of counter examples does not mean proved!

[]
no solution
no solution
no solution
proved
counterexample
[b = [else -> False]]


(ForAll(x, False)) == Not(Exists(x, False))

In [178]:
b = Function("b", Z, B)
prove( ForAll([x], False) == False ) 
prove( Exists([x], False) == False )
prove( ForAll([x], False) ==     Exists([x], False)  )
prove( ForAll([x], False) == Not(Exists([x], False)) )

prove( ForAll([x], b(x) ) == Not(Exists([x], b(x) )) )


proved
proved
proved
counterexample
[]
counterexample
[b = [else -> False]]
